<a href="https://colab.research.google.com/github/aditidas-Projects/information-retrieval-Generative-Relevance-Feedback/blob/main/Information_Retrieval_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-terrier pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Importing all the necessary packages
import pickle
import bs4 as bs
import os
import pyterrier as pt
import pandas as pd
from threading import Thread
import ast
import json
from pyterrier.measures import *
import re
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
pd.set_option('display.max_colwidth', 150)

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



In [ ]:
import json

In [ ]:
folder = 'trec-dl-2019-judged-mine' #folder name to store index
dataset = pt.datasets.get_dataset("irds:msmarco-passage/trec-dl-2019/judged")

In [ ]:
# indexer = pt.IterDictIndexer(f"/content/drive/MyDrive/Trec-DL-2019-Judged/{folder}", overwrite=True, blocks=True) #set your own path
# indexref = indexer.index(dataset.get_corpus_iter())
# index = pt.IndexFactory.of(indexref)

In [ ]:
indexref = pt.IndexRef.of("/content/drive/MyDrive/Trec-DL-2019-Judged/trec-dl-2019-judged-mine/data.properties")
index = pt.IndexFactory.of(indexref)


In [ ]:
print(index)

<org.terrier.structures.Index at 0x78965ff7c900 jclass=org/terrier/structures/Index jself=<LocalRef obj=0x5bf6d52d4c68 at 0x78965fc21f90>>


In [ ]:
qrels = dataset.get_qrels()
topics = dataset.get_topics()
#print(topics)
print("the length of topics is : " , len(topics))

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://trec.nist.gov/data/deep/2019qrels-pass.txt
[INFO] [finished] https://trec.nist.gov/data/deep/2019qrels-pass.txt: [00:00] [187kB] [1.10MB/s]
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [12.1MB/s]
                                                                                                                

the length of topics is :  43


In [ ]:
def get_evaluation(br, topics):
  res = pt.Experiment(
    [br],
    topics,
    qrels,
    baseline=0,
    verbose=True,
    eval_metrics=["map"])
  res_10 = pt.Experiment(
    [br % 10],
    topics,
    qrels,
    baseline=0,
    verbose=True,
    eval_metrics=[RR, "map", nDCG])
  # res_100 = pt.Experiment(
  #     [br % 100],
  #     topics,
  #     qrels,
  #     baseline=0,
  #     verbose=True,
  #     eval_metrics=[RR, "map", nDCG])
  res_1000 = pt.Experiment(
      [br % 1000],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["recall"])
  # print(res)
  # print(f'The value of the evaluation parameters are as follows:\n 1) MRR@10 = {res_10.loc[0, "RR"]}\n 2) MRR@100 = {res_100.loc[0, "RR"]}\n 3) MRR@1000 = {res_1000.loc[0, "RR"]}\n 4) MAP@10 = {res_10.loc[0, "map"]}\n 5) MAP@100 = {res_100.loc[0, "map"]}\n 6) MAP@1000 = {res_1000.loc[0, "map"]}\n 7) nDCG@10 = {res_10.loc[0, "nDCG"]}\n 8) nDCG@100 = {res_100.loc[0, "nDCG"]} \n 9) nDCG@1000 = {res_1000.loc[0, "nDCG"]}')
  print(f'recall@1000 = {res_1000.loc[0, "R@1000"]}')
  print(f'map = {res.loc[0, "map"]}')
  print(f'ndcg@10 = {res_10.loc[0, "nDCG"]}')

In [ ]:
# 2
# Perform retrieval without query expansion using BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : 0.625, "bm25.k_1": 0.625, "bm25.k_3" : 1000})


In [ ]:
print('BM25 without query expansion')
get_evaluation(bm25, topics)

BM25 without query expansion


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7400933766614866
map = 0.3909262170061887
ndcg@10 = 0.2247126674108915


In [ ]:
files = ['/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_document.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_entities.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_entities_cot.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_essay.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_facts.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_keywords.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_keywords_cot.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_news_article.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_queries_keyword.json',
         '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/author_queries/QueryType.QUERY_summary.json']

In [ ]:
def convert_to_json(file):
    with open(file) as f:
        dict_train = json.load(f)
    data_list = []
    for key, value in dict_train.items():
        data_list.append([key, value])

    # Create a DataFrame from the list of lists
    json_df = pd.DataFrame(data_list, columns=['qid', 'query'])

    for itr in range(len(json_df)):
      json_df.loc[itr,'query'] = re.sub('[^0-9a-zA-Z]+', ' ', json_df.loc[itr, 'query'])

    return json_df

In [ ]:
document_by_author = convert_to_json(files[0])
entities_by_author = convert_to_json(files[1])
entities_cot_by_author = convert_to_json(files[2])
essay_by_author = convert_to_json(files[3])
facts_by_author = convert_to_json(files[4])
keywords_by_author = convert_to_json(files[5])
keywords_cot_by_author = convert_to_json(files[6])
news_article_by_author = convert_to_json(files[7])
queries_keyword_by_author = convert_to_json(files[8])
summary_by_author = convert_to_json(files[9])

In [ ]:
# List of DataFrame variables and their corresponding names
dataframes = [
    ('document_by_author', document_by_author),
    ('entities_by_author', entities_by_author),
    ('entities_cot_by_author', entities_cot_by_author),
    ('essay_by_author', essay_by_author),
    ('facts_by_author', facts_by_author),
    ('keywords_by_author', keywords_by_author),
    ('keywords_cot_by_author', keywords_cot_by_author),
    ('news_article_by_author', news_article_by_author),
    ('queries_keyword_by_author', queries_keyword_by_author),
    ('summary_by_author', summary_by_author)
]



In [ ]:
# Iterate through the DataFrames, print their names, and run the evaluation function
for name, df in dataframes:
    print(f"Running evaluation for {name}")
    bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : 0.625, "bm25.k_1": 0.625, "bm25.k_3" : 1000})
    topics = df.copy(deep=True)
    res = pt.Experiment(
      [bm25],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["map"])
    res_10 = pt.Experiment(
      [bm25 % 10],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=[RR, "map", nDCG])
  # res_100 = pt.Experiment(
  #     [bm25 % 100],
  #     topics,
  #     qrels,
  #     baseline=0,
  #     verbose=True,
  #     eval_metrics=[RR, "map", nDCG])
    res_1000 = pt.Experiment(
      [bm25 % 1000],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["recall"])
  # print(res)
  # print(f'The value of the evaluation parameters are as follows:\n 1) MRR@10 = {res_10.loc[0, "RR"]}\n 2) MRR@100 = {res_100.loc[0, "RR"]}\n 3) MRR@1000 = {res_1000.loc[0, "RR"]}\n 4) MAP@10 = {res_10.loc[0, "map"]}\n 5) MAP@100 = {res_100.loc[0, "map"]}\n 6) MAP@1000 = {res_1000.loc[0, "map"]}\n 7) nDCG@10 = {res_10.loc[0, "nDCG"]}\n 8) nDCG@100 = {res_100.loc[0, "nDCG"]} \n 9) nDCG@1000 = {res_1000.loc[0, "nDCG"]}')
    print(f'recall@1000 = {res_1000.loc[0, "R@1000"]}')
    print(f'map = {res.loc[0, "map"]}')
    print(f'ndcg@10 = {res_10.loc[0, "nDCG"]}\n\n\n')

Running evaluation for document_by_author


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7256886534995525
map = 0.3348749689432101
ndcg@10 = 0.2121569455656251



Running evaluation for entities_by_author


...

recall@1000 = 0.7572906457723726
map = 0.4003753856655561
ndcg@10 = 0.21515048913089732



Running evaluation for summary_by_author


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.6594524160889773
map = 0.2931018106550119
ndcg@10 = 0.17721801925843508





In [ ]:
print(document_by_author)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6   1133167   
7    527433   
8   1037798   
9    915593   
10   264014   
11  1121402   
12   962179   
13  1117099   
14   148538   
15   451602   
16   359349   
17  1115776   
18  1112341   
19   131843   
20   104861   
21   833860   
22   207786   
23  1132213   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   287683   
29   490595   
30  1103812   
31    87181   
32   443396   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0                                           Yes goldfish grow They can grow to be quite large depending on the type of goldfish and the size of the tank   
1   Wi Fi and Bluetooth are

In [ ]:
def read_json_and_merge(file, topics_df):
    with open(file) as f:
        dict_train = json.load(f)
    data_list = []
    for key, value in dict_train.items():
        data_list.append([key, value])

    # Create a DataFrame from the list of lists
    json_df = pd.DataFrame(data_list, columns=['qid', 'query_exp'])

    merged_df = pd.merge(topics_df, json_df, on='qid')

    # Concatenate the 'string' columns
    merged_df['query_updated'] = merged_df['query'] + " " + merged_df['query_exp']

    for itr in range(len(merged_df)):
      merged_df.loc[itr,'query_updated'] = re.sub('[^0-9a-zA-Z]+', ' ', merged_df.loc[itr, 'query_updated'])

    # If you want to keep only the 'qid' and combined 'string' columns
    final_df = merged_df[['qid', 'query_updated']]

    # Assuming 'df' is your dataframe
    final_df = final_df.rename(columns={'qid': 'qid', 'query_updated': 'query'})

    return final_df

In [ ]:
topics = dataset.get_topics()
#we now evaluate values by combining each subtask with original topics
topics_and_document = read_json_and_merge(files[0], topics)
topics_and_entities = read_json_and_merge(files[1], topics)
topics_and_entities_cot = read_json_and_merge(files[2], topics)
topics_and_essay = read_json_and_merge(files[3], topics)
topics_and_facts = read_json_and_merge(files[4], topics)
topics_and_keywords = read_json_and_merge(files[5], topics)
topics_and_keywords_cot = read_json_and_merge(files[6], topics)
topics_and_news_article = read_json_and_merge(files[7], topics)
topics_and_queries_keyword = read_json_and_merge(files[8], topics)
topics_and_summary = read_json_and_merge(files[9], topics)

In [ ]:
dataframes_with_topics = [
    ('topics_and_document', topics_and_document),
    ('topics_and_entities', topics_and_entities),
    ('topics_and_entities_cot', topics_and_entities_cot),
    ('topics_and_essay', topics_and_essay),
    ('topics_and_facts', topics_and_facts),
    ('topics_and_keywords', topics_and_keywords),
    ('topics_and_keywords_cot', topics_and_keywords_cot),
    ('topics_and_news_article', topics_and_news_article),
    ('topics_and_queries_keyword', topics_and_queries_keyword),
    ('topics_and_summary', topics_and_summary)
]


In [ ]:
# Iterate through the DataFrames, print their names, and run the evaluation function
for name, df in dataframes_with_topics:
    print(f"Running evaluation for {name}")
    bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : 0.625, "bm25.k_1": 0.625, "bm25.k_3" : 1000})
    topics = df.copy(deep=True)
    res = pt.Experiment(
      [bm25],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["map"])
    res_10 = pt.Experiment(
      [bm25 % 10],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=[RR, "map", nDCG])
  # res_100 = pt.Experiment(
  #     [bm25 % 100],
  #     topics,
  #     qrels,
  #     baseline=0,
  #     verbose=True,
  #     eval_metrics=[RR, "map", nDCG])
    res_1000 = pt.Experiment(
      [bm25 % 1000],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["recall"])
  # print(res)
  # print(f'The value of the evaluation parameters are as follows:\n 1) MRR@10 = {res_10.loc[0, "RR"]}\n 2) MRR@100 = {res_100.loc[0, "RR"]}\n 3) MRR@1000 = {res_1000.loc[0, "RR"]}\n 4) MAP@10 = {res_10.loc[0, "map"]}\n 5) MAP@100 = {res_100.loc[0, "map"]}\n 6) MAP@1000 = {res_1000.loc[0, "map"]}\n 7) nDCG@10 = {res_10.loc[0, "nDCG"]}\n 8) nDCG@100 = {res_100.loc[0, "nDCG"]} \n 9) nDCG@1000 = {res_1000.loc[0, "nDCG"]}')
    print(f'recall@1000 = {res_1000.loc[0, "R@1000"]}')
    print(f'map = {res.loc[0, "map"]}')
    print(f'ndcg@10 = {res_10.loc[0, "nDCG"]}\n\n\n')

Running evaluation for topics_and_document


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7882992338978946
map = 0.43904890962134085
ndcg@10 = 0.24222054638934393



Running evaluation for topics_and_entities


...

recall@1000 = 0.7930166312271975
map = 0.42157916152116776
ndcg@10 = 0.2241366722807144



Running evaluation for topics_and_summary


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7759611337492336
map = 0.3801710761146841
ndcg@10 = 0.2167171167699835





In [ ]:
print(topics_and_document)

In [ ]:
# topics = dataset.get_topics()

# our_essay = read_json_and_merge("/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_essay.json", topics)

In [ ]:
# print(our_essay)

In [ ]:
# print('Output for our essay')
# get_evaluation(bm25, our_essay)

Output for our essay


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7828308335199505
map = 0.4266597603598687
ndcg@10 = 0.25839946564971383


In [ ]:
folder = "/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Final_Input_Queries"

In [ ]:
topics = dataset.get_topics()
df_documents = topics.copy(deep = True)

with open(f'{folder}/query_document.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_documents.loc[it, 'query'] += ' ' + str(ls[k]['relevant_documents'])
  it += 1

for it in range(len(df_documents)):
  df_documents.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_documents.loc[it, 'query'])

In [ ]:
print(len(df_documents))

43


In [ ]:
print('Output for our documents')
get_evaluation(bm25, df_documents)

Output for our documents


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8182601161947405
map = 0.4513110310001279
ndcg@10 = 0.25619656273046126


In [ ]:
topics = dataset.get_topics()
df_entities = topics.copy(deep = True)

with open(f'{folder}/query_entities.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_entities.loc[it, 'query'] += ' ' + str(ls[k]['relevant_entities'])
  it += 1

for it in range(len(df_entities)):
  df_entities.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_entities.loc[it, 'query'])

In [ ]:
print(df_entities)

In [ ]:
print('Output for our entities')
get_evaluation(bm25, df_entities)

Output for our entities


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7935332430515921
map = 0.43765747265999333
ndcg@10 = 0.25490205333472404


In [ ]:
topics = dataset.get_topics()
df_entities_cot = topics.copy(deep = True)

with open(f'{folder}/query_entities_cot.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_entities_cot.loc[it, 'query'] += ' ' + str(ls[k]['relevant_entities'])
  it += 1

for it in range(len(df_entities_cot)):
  df_entities_cot.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_entities_cot.loc[it, 'query'])

In [ ]:
print(df_entities_cot)

In [ ]:
print('Output for our entities_cot')
get_evaluation(bm25, df_entities_cot)

Output for our entities_cot


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7531534617811949
map = 0.34392542382296437
ndcg@10 = 0.225956589631738


In [ ]:
topics = dataset.get_topics()
df_essay = topics.copy(deep = True)

with open(f'{folder}/query_essay.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_essay.loc[it, 'query'] += ' ' + str(ls[k]['essay'])
  it += 1

for it in range(len(df_essay)):
  df_essay.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_essay.loc[it, 'query'])

In [ ]:
print(df_essay)

In [ ]:
print('Output for our essay')
get_evaluation(bm25, df_essay)

Output for our essay


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7828308335199505
map = 0.4266597603598687
ndcg@10 = 0.25839946564971383


In [ ]:
topics = dataset.get_topics()
df_facts = topics.copy(deep = True)

with open(f'{folder}/query_facts.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_facts.loc[it, 'query'] += ' ' + str(ls[k]['relevant_facts'])
  it += 1

for it in range(len(df_facts)):
  df_facts.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_facts.loc[it, 'query'])

In [ ]:
print(df_facts)

In [ ]:
print('Output for our facts')
get_evaluation(bm25, df_facts)

Output for our facts


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8050821768593277
map = 0.4185793033601474
ndcg@10 = 0.25200537247985055


In [ ]:
topics = dataset.get_topics()
df_keywords = topics.copy(deep = True)

with open(f'{folder}/query_keywords.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_keywords.loc[it, 'query'] += ' ' + str(ls[k]['keywords'])
  it += 1

for it in range(len(df_keywords)):
  df_keywords.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_keywords.loc[it, 'query'])

In [ ]:
print(df_keywords)

In [ ]:
print('Output for our keywords')
get_evaluation(bm25, df_keywords)

Output for our keywords


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7878392928856607
map = 0.4057043622819042
ndcg@10 = 0.23059148342017485


In [ ]:
topics = dataset.get_topics()
df_keywords_cot = topics.copy(deep = True)

with open(f'{folder}/query_keywords_cot.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_keywords_cot.loc[it, 'query'] += ' ' + str(ls[k]['relevant_keywords'])
  it += 1

for it in range(len(df_keywords_cot)):
  df_keywords_cot.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_keywords_cot.loc[it, 'query'])

In [ ]:
print(df_keywords_cot)

In [ ]:
print('Output for our keywords_cot')
get_evaluation(bm25, df_keywords_cot)

Output for our keywords_cot


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7587003437383076
map = 0.3541276395671401
ndcg@10 = 0.22150843219986205


In [ ]:
topics = dataset.get_topics()
df_keywords_diverse = topics.copy(deep = True)

with open(f'{folder}/query_keywords_diverse.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_keywords_diverse.loc[it, 'query'] += ' ' + str(ls[k])
  it += 1

for it in range(len(df_keywords_diverse)):
  df_keywords_diverse.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_keywords_diverse.loc[it, 'query'])

In [ ]:
print(df_keywords_diverse)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow Goldfish growth stages Factors influencing goldfish size Goldfish care for optimal growth Environmental impact on goldfish develo...  
1   what is wifi vs bluetoo

In [ ]:
print('Output for our keywords_diverse')
get_evaluation(bm25, df_keywords_diverse)

Output for our keywords_diverse


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7888780882507906
map = 0.3847290879115059
ndcg@10 = 0.2288134303577298


In [ ]:
topics = dataset.get_topics()
df_news = topics.copy(deep = True)

with open(f'{folder}/query_news.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_news.loc[it, 'query'] += ' ' + str(ls[k]['article'])
  it += 1

for it in range(len(df_news)):
  df_news.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_news.loc[it, 'query'])

In [ ]:
print(df_news)

In [ ]:
print('Output for our news')
get_evaluation(bm25, df_news)

Output for our news


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.797873571177627
map = 0.43565597399225625
ndcg@10 = 0.25979699896826464


In [ ]:
topics = dataset.get_topics()
df_summary = topics.copy(deep = True)

with open(f'{folder}/query_summary.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_summary.loc[it, 'query'] += ' ' + str(ls[k]['summary'])
  it += 1

for it in range(len(df_summary)):
  df_summary.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_summary.loc[it, 'query'])

In [ ]:
print(df_summary)

In [ ]:
print('Output for our summary')
get_evaluation(bm25, df_summary)

Output for our summary


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7923334315475553
map = 0.43048831027896806
ndcg@10 = 0.25853291308882875


In [ ]:
topics = dataset.get_topics()
df_essay_bard = topics.copy(deep = True)

with open(f'{folder}/essay_bard.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_essay_bard.loc[it, 'query'] += ' ' + str(ls[k])
  it += 1

for it in range(len(df_essay_bard)):
  df_essay_bard.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_essay_bard.loc[it, 'query'])

  df_aditi_essay = df_essay_bard

In [ ]:
print(len(df_aditi_essay))

43


In [ ]:
print('Output for my aditi essay')
get_evaluation(bm25, df_aditi_essay)

Output for my aditi essay


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

KeyboardInterrupt: ignored

In [ ]:
topics = dataset.get_topics()
df_expanded_queries = topics.copy(deep = True)

with open(f'{folder}/expanded_queries.json') as f:
  ls = json.load(f)

for it in range(len(df_expanded_queries)):
  syn_list = ls['query'][str(it)]['expanded']
  syn_str = ''

  for syn in syn_list:
    syn_str += syn + ' '

  df_expanded_queries.loc[it, 'query'] += ' ' + ls['query'][str(it)]['original'] + ' ' + ls['query'][str(it)]['expanded']

for it in range(len(df_expanded_queries)):
  df_expanded_queries.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_expanded_queries.loc[it, 'query'])

In [ ]:
print(df_expanded_queries)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0                                                                           do goldfish grow do goldfish grow Expand upon the growth process of goldfish   
1                          

In [ ]:
print('Output for my expanded queries')
get_evaluation(bm25, df_expanded_queries)

Output for my expanded queries


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7477550396176796
map = 0.40529293710854186
ndcg@10 = 0.23408133779035376


In [ ]:
topics = dataset.get_topics()
df_web_page = topics.copy(deep = True)

with open(f'{folder}/web_page.json') as f:
  ls = json.load(f)

for it in range(len(df_web_page)):
  syn_list = ls['query'][str(it)]['content']
  syn_str = ''

  for syn in syn_list:
    syn_str += syn + ' '

  df_web_page.loc[it, 'query'] += ' ' + ls['query'][str(it)]['title'] + ' ' + ls['query'][str(it)]['content']

for it in range(len(df_web_page)):
  df_web_page.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_web_page.loc[it, 'query'])

In [ ]:
print(df_web_page)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow Understanding Goldfish Growth Goldfish like many fish go through a growth process Learn about the factors influencing goldfish gr...  
1   what is wifi vs bluetoo

In [ ]:
print('Output for my web page queries')
get_evaluation(bm25, df_web_page)

Output for my web page queries


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7732840756093897
map = 0.4050404611048009
ndcg@10 = 0.24923673649355063


In [ ]:
topics = dataset.get_topics()
df_domain_terms = topics.copy(deep = True)

with open(f'{folder}/domain_terms.json') as f:
  ls = json.load(f)

  for it in range(len(df_domain_terms)):
    df_domain_terms.loc[it, 'query'] += ' ' + str(ls['query'][str(it)])

  for it in range(len(df_domain_terms)):
    df_domain_terms.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_domain_terms.loc[it, 'query'])

In [ ]:
print(df_domain_terms)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow goldfish fish growth aquatic pets tank maintenance feeding habits development stages fish care aquariums pet ownership goldfish l...  
1   what is wifi vs bluetoo

In [ ]:
print('Output for my domain terns queries')
get_evaluation(bm25, df_domain_terms)

Output for my domain terns queries


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.6733306187440931
map = 0.242949350343714
ndcg@10 = 0.1558882848059561


In [ ]:
topics = dataset.get_topics()
df_closely_related = topics.copy(deep = True)

with open(f'{folder}/closely_related.json') as f:
  ls = json.load(f)

for it in range(len(df_closely_related)):
  df_closely_related.loc[it, 'query'] += ' ' + str(ls['query'][str(it)])

for it in range(len(df_closely_related)):
  df_closely_related.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_closely_related.loc[it, 'query'])

In [ ]:
print(df_closely_related)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow goldfish growth stages goldfish size how fast do goldfish grow goldfish care tips aquarium fish growth goldfish development optim...  
1   what is wifi vs bluetoo

In [ ]:
print('Output for my closely related queries')
get_evaluation(bm25, df_closely_related)

Output for my closely related queries


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7854663743498564
map = 0.37210264318632874
ndcg@10 = 0.20732121548535637


In [ ]:
# our_files = ['/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_document.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_entities.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_entities_cot.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_essay.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_facts.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_keywords.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_keywords_cot.json',
#              '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_news.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_keywords_diverse.json',
#             '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/Hemang_outputs/query_summary.json']

In [ ]:
# our_document = read_json_and_merge(our_files[0], topics)
# our_entities = read_json_and_merge(our_files[1], topics)
# our_entities_cot = read_json_and_merge(our_files[2], topics)
# our_essay = read_json_and_merge(our_files[3], topics)
# our_facts = read_json_and_merge(our_files[4], topics)
# our_keywords = read_json_and_merge(our_files[5], topics)
# our_keywords_cot = read_json_and_merge(our_files[6], topics)
# our_news_article = read_json_and_merge(our_files[7], topics)
# our_queries_keyword = read_json_and_merge(our_files[8], topics)
# our_summary = read_json_and_merge(our_files[9], topics)


In [ ]:
# our_dataframes_with_topics = [
#     ('our_document', our_document),
#     ('our_entities', our_entities),
#     ('our_entities_cot', our_entities_cot),
#     ('our_essay', our_essay),
#     ('our_facts', our_facts),
#     ('our_keywords', our_keywords),
#     ('our_keywords_cot', our_keywords_cot),
#     ('our_news_article', our_news_article),
#     ('our_queries_keyword', our_queries_keyword),
#     ('our_summary', our_summary)
# ]

In [ ]:
# json_topics = topics.to_json()
# with open('/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/output.json', 'w') as json_file:
#     json_file.write(json_topics)

In [ ]:
df_alt_sentences = topics.copy(deep = True)

with open(f'/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/modified-topics/alternate-sentences.json') as f:
  ls = json.load(f)

for it in range(len(df_alt_sentences)):
  alt_sentences_list = ls['query'][str(it)]
  alt_sentences_str = ''

  for alt_sentences in alt_sentences_list:
    alt_sentences_str += alt_sentences + ' '

  df_alt_sentences.loc[it, 'query'] += ' ' + alt_sentences_str

for it in range(len(df_alt_sentences)):
  df_alt_sentences.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_alt_sentences.loc[it, 'query'])

In [ ]:
print(df_alt_sentences)

        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow Explore the intricate process of goldfish growth considering factors such as diet environment and genetic influences Investigate ...  
1   what is wifi vs bluetoo

In [ ]:
get_evaluation(bm25, df_alt_sentences)

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8140588913579707
map = 0.44008172547745217
ndcg@10 = 0.2439635515009891


In [ ]:
def get_evaluation2(br, topics):
  res = pt.Experiment(
    [br],
    topics,
    qrels,
    baseline=0,
    verbose=True,
    eval_metrics=["map"])
  res_10 = pt.Experiment(
    [br % 10],
    topics,
    qrels,
    baseline=0,
    verbose=True,
    eval_metrics=[RR, "map", nDCG])
  # res_100 = pt.Experiment(
  #     [br % 100],
  #     topics,
  #     qrels,
  #     baseline=0,
  #     verbose=True,
  #     eval_metrics=[RR, "map", nDCG])
  res_1000 = pt.Experiment(
      [br % 1000],
      topics,
      qrels,
      baseline=0,
      verbose=True,
      eval_metrics=["recall"])
  # print(res)
  # print(f'The value of the evaluation parameters are as follows:\n 1) MRR@10 = {res_10.loc[0, "RR"]}\n 2) MRR@100 = {res_100.loc[0, "RR"]}\n 3) MRR@1000 = {res_1000.loc[0, "RR"]}\n 4) MAP@10 = {res_10.loc[0, "map"]}\n 5) MAP@100 = {res_100.loc[0, "map"]}\n 6) MAP@1000 = {res_1000.loc[0, "map"]}\n 7) nDCG@10 = {res_10.loc[0, "nDCG"]}\n 8) nDCG@100 = {res_100.loc[0, "nDCG"]} \n 9) nDCG@1000 = {res_1000.loc[0, "nDCG"]}')
  output_array = []
  print(f'recall@1000 = {res_1000.loc[0, "R@1000"]}')
  print(f'map = {res.loc[0, "map"]}')
  print(f'ndcg@10 = {res_10.loc[0, "nDCG"]}')
  output_array.append(f'recall@1000 = {res_1000.loc[0, "R@1000"]}')
  output_array.append(f'map = {res.loc[0, "map"]}')
  output_array.append(f'ndcg@10 = {res_10.loc[0, "nDCG"]}')

  return output_array


In [ ]:
from itertools import combinations
store_pnc_metrics = []
labels = []

def comb(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)

  for x in all_comb:
    print(x)
    labels.append(x)
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    store_pnc_metrics.append(get_evaluation2(bm25, df_comb))

ls = {
    'df_alt_sentences': df_alt_sentences,
    'df_documents': df_documents,
    'df_entities': df_entities,
    'df_news': df_news,
    #'df_context_rel': df_context_rel,
}

for it in range(len(ls),len(ls)+1):
  comb(ls, it)

('df_alt_sentences', 'df_documents', 'df_entities', 'df_news')


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8519664869998683
map = 0.5231208599372107
ndcg@10 = 0.2915071938843527


In [ ]:
print(labels)

[('df_alt_sentences', 'df_documents', 'df_entities', 'df_news')]


In [ ]:
print(store_pnc_metrics)

[['recall@1000 = 0.8519664869998683', 'map = 0.5231208599372107', 'ndcg@10 = 0.2915071938843527']]


In [ ]:
# Your data
keys = [('df_alt_sentences', 'df_documents'),
        ('df_alt_sentences', 'df_entities'),
        ('df_alt_sentences', 'df_news'),
        ('df_documents', 'df_entities'),
        ('df_documents', 'df_news'),
        ('df_entities', 'df_news'),
        ('df_alt_sentences', 'df_documents', 'df_entities'),
        ('df_alt_sentences', 'df_documents', 'df_news'),
        ('df_alt_sentences', 'df_entities', 'df_news'),
        ('df_documents', 'df_entities', 'df_news'),
        ('df_alt_sentences', 'df_documents', 'df_entities', 'df_news')]

values = [['recall@1000 = 0.861385948656575', 'map = 0.5195342168979958', 'ndcg@10 = 0.28644676394513'],
          ['recall@1000 = 0.824162678012577', 'map = 0.4922487036187653', 'ndcg@10 = 0.2766549379563848'],
          ['recall@1000 = 0.8236443218520734', 'map = 0.4850485513129176', 'ndcg@10 = 0.2718889756273727'],
          ['recall@1000 = 0.8524900818892379', 'map = 0.5188019292396547', 'ndcg@10 = 0.2881305693967742'],
          ['recall@1000 = 0.8522597531875307', 'map = 0.5052762203122478', 'ndcg@10 = 0.2840249680676692'],
          ['recall@1000 = 0.8262445441870947', 'map = 0.49192233881415637', 'ndcg@10 = 0.28512004997947143'],
          ['recall@1000 = 0.8592934192928631', 'map = 0.5251726339307353', 'ndcg@10 = 0.2914346692159699'],
          ['recall@1000 = 0.8550205048741225', 'map = 0.5179859925557759', 'ndcg@10 = 0.2856524561455522'],
          ['recall@1000 = 0.8267700302546064', 'map = 0.4970447053313532', 'ndcg@10 = 0.2766122853683651'],
          ['recall@1000 = 0.8501948867694927', 'map = 0.5184883764094708', 'ndcg@10 = 0.29283921745953323'],
          ['recall@1000 = 0.8519664869998683', 'map = 0.5231208599372107', 'ndcg@10 = 0.2915071938843527']]

# Create a dictionary to map keys to values
mapping_dict = dict(zip(keys, values))

# Print the dictionary
print(mapping_dict)


{('df_alt_sentences', 'df_documents'): ['recall@1000 = 0.861385948656575', 'map = 0.5195342168979958', 'ndcg@10 = 0.28644676394513'], ('df_alt_sentences', 'df_entities'): ['recall@1000 = 0.824162678012577', 'map = 0.4922487036187653', 'ndcg@10 = 0.2766549379563848'], ('df_alt_sentences', 'df_news'): ['recall@1000 = 0.8236443218520734', 'map = 0.4850485513129176', 'ndcg@10 = 0.2718889756273727'], ('df_documents', 'df_entities'): ['recall@1000 = 0.8524900818892379', 'map = 0.5188019292396547', 'ndcg@10 = 0.2881305693967742'], ('df_documents', 'df_news'): ['recall@1000 = 0.8522597531875307', 'map = 0.5052762203122478', 'ndcg@10 = 0.2840249680676692'], ('df_entities', 'df_news'): ['recall@1000 = 0.8262445441870947', 'map = 0.49192233881415637', 'ndcg@10 = 0.28512004997947143'], ('df_alt_sentences', 'df_documents', 'df_entities'): ['recall@1000 = 0.8592934192928631', 'map = 0.5251726339307353', 'ndcg@10 = 0.2914346692159699'], ('df_alt_sentences', 'df_documents', 'df_news'): ['recall@1000 =

In [ ]:
from itertools import combinations
store_pnc_metrics_gpt = []
labels = []

def comb(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)

  for x in all_comb:
    print(x)
    labels.append(x)
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    store_pnc_metrics_gpt.append(get_evaluation2(bm25 >> pt.rewrite.RM3(index) >>bm25, df_comb))
    #print(len(df_comb.loc[42, 'query']))

ls = {
    'df_alt_sentences': df_alt_sentences,
    'df_documents': df_documents,
    'df_essay': df_essay,
    'df_news': df_news,
    'df_summary' : df_summary,
    'df_entities' : df_entities,
    'df_entities_cot' : df_entities_cot,
    'df_closely_related' : df_closely_related,
    'df_expanded_queries' : df_expanded_queries,
    'df_facts' : df_facts,
    'df_keywords': df_keywords,
    'df_keywords_cot': df_keywords_cot,
    'df_keywords_diverse': df_keywords_diverse,
    'df_web_page' : df_web_page,
    #'df_context_rel': df_context_rel,
}
#print(len(df_aditi_essay.loc[0, 'query']) + len(df_documents.loc[0, 'query']) + len(df_alt_sentences.loc[0, 'query']))
for it in range(len(ls), len(ls) + 1):
  comb(ls, it)

# ans = 0
# for it in ls.keys():
#   ans += len(ls[it].loc[42,'query'])


#print(ans)

('df_alt_sentences', 'df_documents', 'df_essay', 'df_news', 'df_summary', 'df_entities', 'df_entities_cot', 'df_closely_related', 'df_expanded_queries', 'df_facts', 'df_keywords', 'df_keywords_cot', 'df_keywords_diverse', 'df_web_page')


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8231206706261271
map = 0.48279790147794527
ndcg@10 = 0.2714785864559992


In [ ]:
print(store_pnc_metrics_gpt)

[['recall@1000 = 0.8433900331427368', 'map = 0.5120845881828806', 'ndcg@10 = 0.28342332467402187']]


In [ ]:
df_alt_sentences_bard = topics.copy(deep = True)

with open(f'/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/modified-topics/alternate_sentences_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_alt_sentences_bard)):
  alt_sentences_list = ls['query'][str(it)]
  alt_sentences_str = ''

  for alt_sentences in alt_sentences_list:
    alt_sentences_str += alt_sentences + ' '

  df_alt_sentences_bard.loc[it, 'query'] += ' ' + alt_sentences_str

for it in range(len(df_alt_sentences_bard)):
  df_alt_sentences_bard.loc[it, 'query'] = re.sub('[^0-9a-zA-Z]+', ' ', df_alt_sentences_bard.loc[it, 'query'])

In [ ]:
print(df_alt_sentences_bard)


        qid  \
0    156493   
1   1110199   
2   1063750   
3    130510   
4    489204   
5    573724   
6    168216   
7   1133167   
8    527433   
9   1037798   
10   915593   
11   264014   
12  1121402   
13   962179   
14  1117099   
15   148538   
16   451602   
17   359349   
18  1115776   
19  1112341   
20   131843   
21   104861   
22   833860   
23   207786   
24  1114819   
25   183378   
26  1106007   
27  1124210   
28   490595   
29  1103812   
30    87181   
31   443396   
32  1121709   
33    87452   
34   855410   
35  1114646   
36    19335   
37    47923   
38   405717   
39   182539   
40  1113437   
41  1129237   
42   146187   

                                                                                                                                                    query  
0   do goldfish grow Do goldfish continue to grow throughout their lives How big do goldfish typically grow What are the factors that influence the gr...  
1   what is wifi vs bluetoo

In [ ]:
# bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"bm25.b" : 0.625, "bm25.k_1": 0.625})

get_evaluation(bm25, df_alt_sentences_bard)


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7764276565698082
map = 0.41212957481411705
ndcg@10 = 0.22820762576334824


In [ ]:
folder_bard = '/content/drive/MyDrive/Trec-DL-2019-Judged/IR_indexes/BARD_Input_Queries'

In [ ]:
topics = dataset.get_topics()
df_cot_entities_bard = topics.copy(deep = True)

with open(f'{folder_bard}/cot_entities_bard.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_cot_entities_bard.loc[it, 'query'] += ' ' + str(ls[k]['chain_of_thought'])
  it += 1

for it in range(len(df_cot_entities_bard)):
  df_cot_entities_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_cot_entities_bard.loc[it, 'query'])

topics = dataset.get_topics()
df_diverse_keywords_bard = topics.copy(deep = True)

with open(f'{folder_bard}/diverse_keywords_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_diverse_keywords_bard)):
  df_diverse_keywords_bard.loc[it, 'query'] += ' ' + str(ls['queries'][str(it)]['keywords'])

for it in range(len(df_diverse_keywords_bard)):
  df_diverse_keywords_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_diverse_keywords_bard.loc[it, 'query'])

topics = dataset.get_topics()
df_keywords_bard = topics.copy(deep = True)

with open(f'{folder_bard}/keywords_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_keywords_bard)):
  df_keywords_bard.loc[it, 'query'] += ' ' + str(ls[str(it)]['keywords'])

for it in range(len(df_keywords_bard)):
  df_keywords_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_keywords_bard.loc[it, 'query'])

topics = dataset.get_topics()
df_rel_entities_bard = topics.copy(deep = True)

with open(f'{folder_bard}/relevannt_entities_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_rel_entities_bard)):
  df_rel_entities_bard.loc[it, 'query'] += ' ' + str(ls[str(it)]['terms'])

for it in range(len(df_rel_entities_bard)):
  df_rel_entities_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_rel_entities_bard.loc[it, 'query'])


In [ ]:
topics = dataset.get_topics()
df_cot_keywords_bard = topics.copy(deep = True)


with open(f'{folder_bard}/cot_keywords_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_cot_keywords_bard)):
  df_cot_keywords_bard.loc[it, 'query'] += ' ' + str(ls[str(it)]['relevant_keywords'])

for it in range(len(df_cot_keywords_bard)):
  df_cot_keywords_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_cot_keywords_bard.loc[it, 'query'])

In [ ]:
topics = dataset.get_topics()
df_summary_bard = topics.copy(deep = True)

with open(f'{folder_bard}/summary_bard.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_summary_bard.loc[it, 'query'] += ' ' + str(ls[k]['summary'])
  it += 1

for it in range(len(df_summary_bard)):
  df_summary_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_summary_bard.loc[it, 'query'])

In [ ]:
topics = dataset.get_topics()
df_facts_bard = topics.copy(deep = True)

with open(f'{folder_bard}/facts_bard.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_facts_bard.loc[it, 'query'] += ' ' + str(ls[k]['facts'])
  it += 1

for it in range(len(df_facts_bard)):
  df_facts_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_facts_bard.loc[it, 'query'])

In [ ]:
topics = dataset.get_topics()
df_document_bard = topics.copy(deep = True)

with open(f'{folder_bard}/document_bard.json') as f:
  ls = json.load(f)

it = 0
for k in ls.keys():
  df_document_bard.loc[it, 'query'] += ' ' + str(ls[k]['document'])
  it += 1

for it in range(len(df_facts_bard)):
  df_document_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_document_bard.loc[it, 'query'])

In [ ]:
topics = dataset.get_topics()
df_news_bard = topics.copy(deep = True)

with open(f'{folder_bard}/news_bard.json') as f:
  ls = json.load(f)

for it in range(len(df_news_bard)):
  df_news_bard.loc[it, 'query'] += ' ' + str(ls[str(it)]['news'])

for it in range(len(df_news_bard)):
  df_news_bard.loc[it, 'query'] = re.sub('[^0-9.a-zA-Z]+', ' ', df_news_bard.loc[it, 'query'])

In [ ]:
print("for document")
get_evaluation(bm25,df_document_bard)


print("\nfor news")
get_evaluation(bm25,df_news_bard)


print("\nfor facts")
get_evaluation(bm25,df_facts_bard)


print("\nfor summary")
get_evaluation(bm25,df_summary_bard)


print("\nfor cot-keywords")
get_evaluation(bm25,df_cot_keywords_bard)


print("\nfor entities")
get_evaluation(bm25,df_rel_entities_bard)


print("\nfor keywords")
get_evaluation(bm25,df_keywords_bard)


print("\nfor diverse-keywords aka queries")
get_evaluation(bm25,df_diverse_keywords_bard)


print("\nfor cot-entities")
get_evaluation(bm25,df_cot_entities_bard)

for document


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8320393156353618
map = 0.4828855026679164
ndcg@10 = 0.2598004492635481

for news


...

recall@1000 = 0.76573431105288
map = 0.4001873380626954
ndcg@10 = 0.21932952444904774

for cot-entities


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8175085468211093
map = 0.4409142056343354
ndcg@10 = 0.23826064805856284


In [ ]:
from itertools import combinations
store_pnc_metrics_bard = []
labels = []

def comb(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)

  for x in all_comb:
    print(x)
    labels.append(x)
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    store_pnc_metrics_bard.append(get_evaluation2(bm25 >> pt.rewrite.RM3(index) >>bm25, df_comb))

ls = {
    #'df_alt_sentences': df_alt_sentences,
    'df_document_bard': df_document_bard, #recall : 0.83
    'df_facts_bard': df_facts_bard, #recall : 0.82
    'df_news_bard': df_news_bard, #map : 0.449
    'df_summary_bard': df_summary_bard, #map : 0.458
    'df_essay_bard' : df_essay_bard,
    'df_cot_keywords_bard' : df_cot_keywords_bard,
    'df_rel_entities_bard' : df_rel_entities_bard,
    'df_keywords_bard' : df_keywords_bard,
    'df_cot_entities_bard' : df_cot_entities_bard,
    'df_diverse_keywords_bard' : df_diverse_keywords_bard,
    #'df_context_rel': df_context_rel,
}

for it in range(len(ls), len(ls)+1):
  comb(ls, it)

('df_document_bard', 'df_facts_bard', 'df_news_bard', 'df_summary_bard', 'df_essay_bard', 'df_cot_keywords_bard', 'df_rel_entities_bard', 'df_keywords_bard', 'df_cot_entities_bard', 'df_diverse_keywords_bard')


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8376678018276568
map = 0.5257776326437188
ndcg@10 = 0.2877766329096955


In [ ]:
print(labels)
print(store_pnc_metrics_bard)

[('df_document_bard', 'df_facts_bard', 'df_news_bard', 'df_summary_bard', 'df_essay_bard', 'df_cot_keywords_bard', 'df_rel_entities_bard', 'df_keywords_bard', 'df_cot_entities_bard', 'df_diverse_keywords_bard')]
[['recall@1000 = 0.8745587108142275', 'map = 0.552123290161896', 'ndcg@10 = 0.2989024144054825']]


In [ ]:
#top - 4 recall
from itertools import combinations
store_combined_metrics_bard_gpt_recall = []
labels = []

def comb(ls, num_of_permutations):
  all_comb = combinations(ls.keys(), num_of_permutations)

  for x in all_comb:
    print(x)
    labels.append(x)
    df_comb = topics.copy(deep = True)
    for k in x:
      for it in range(len(df_comb)):
        df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

    store_combined_metrics_bard_gpt_recall.append(get_evaluation2(bm25, df_comb))

ls = {
    #'df_alt_sentences': df_alt_sentences,
    'df_document_bard': df_document_bard, #recall : 0.83
    'df_facts_bard': df_facts_bard, #recall : 0.82
    'df_alt_sentences_gpt': df_alt_sentences,
    'df_documents_gpt': df_documents,
    #'df_context_rel': df_context_rel,
}

for it in range(len(ls), len(ls)+1):
  comb(ls, it)

('df_document_bard', 'df_facts_bard', 'df_alt_sentences_gpt', 'df_documents_gpt')


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8672790400226514
map = 0.5580793029526988
ndcg@10 = 0.304934125999687


In [ ]:
print(labels)
print(store_combined_metrics_bard_gpt_recall)

[('df_document_bard', 'df_facts_bard', 'df_alt_sentences_gpt', 'df_documents_gpt')]
[['recall@1000 = 0.8672790400226514', 'map = 0.5580793029526988', 'ndcg@10 = 0.304934125999687']]


In [ ]:
print("This is for BARD [BM25 + RM3]\n\n")
print("for document")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_document_bard)


print("\nfor news")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_news_bard)


print("\nfor facts")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_facts_bard)


print("\nfor summary")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_summary_bard)


print("\nfor cot-keywords")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_cot_keywords_bard)


print("\nfor entities")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_rel_entities_bard)


print("\nfor keywords")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_keywords_bard)


print("\nfor diverse-keywords aka queries")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_diverse_keywords_bard)


print("\nfor cot-entities")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_cot_entities_bard)

This is for BARD [BM25 + RM3]


for document


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8297154665452284
map = 0.4858009917602869
ndcg@10 = 0.25085975197412974

for news


...

recall@1000 = 0.8034631373740964
map = 0.4324284224305867
ndcg@10 = 0.21934902389589675

for cot-entities


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8043938845687808
map = 0.441241911170771
ndcg@10 = 0.22606215468605417


In [ ]:
print("\nfor essay")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >> bm25, df_essay_bard)



for essay


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7937520340681454
map = 0.4602336918160405
ndcg@10 = 0.26296838432023895


In [ ]:
#get_evaluation(bm25 >> pt.rewrite.RM3(index) >> bm25, df_aditi_essay)
print("This is for GPT-3 [BM25 + RM3]\n\n")
# print("for document")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_documents)


# print("\nfor news")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_news)


# print("\nfor facts")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_facts)


# print("\nfor summary")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_summary)


# print("\nfor cot-keywords")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_keywords_cot)


# print("\nfor entities")
# get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_entities)


print("\nfor keywords")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_keywords)


print("\nfor diverse-keywords aka queries")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_keywords_diverse)


print("\nfor cot-entities")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,df_entities_cot)


print("\nfor essay")
get_evaluation(bm25 >> pt.rewrite.RM3(index) >> bm25, df_essay)


This is for GPT-3 [BM25 + RM3]



for keywords


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7913032157592879
map = 0.4296537306265953
ndcg@10 = 0.2299335030714261

for diverse-keywords aka queries


...

recall@1000 = 0.7769471260506188
map = 0.39478678399762396
ndcg@10 = 0.2209912255661453

for essay


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7911638350046318
map = 0.46738887379869054
ndcg@10 = 0.2578791208742252


In [ ]:
# #top - 4 map
# from itertools import combinations
# store_combined_metrics_bard_gpt_recall = []
# labels = []

# def comb(ls, num_of_permutations):
#   all_comb = combinations(ls.keys(), num_of_permutations)

#   for x in all_comb:
#     print(x)
#     labels.append(x)
#     df_comb = topics.copy(deep = True)
#     for k in x:
#       for it in range(len(df_comb)):
#         df_comb.loc[it, 'query'] += ' ' + ls[k].loc[it, 'query']

#     store_combined_metrics_bard_gpt_recall.append(get_evaluation2(bm25, df_comb))

# ls = {
#     #'df_alt_sentences': df_alt_sentences,
#     'df_document_bard': df_document_bard, #recall : 0.83
#     'df_facts_bard' : df_facts_bard, #recall : 0.82
#     'df_alt_sentences_gpt' : df_facts,
#     'df_documents_gpt': df_documents,
#     #'df_context_rel': df_context_rel,
# }

# for it in range(len(ls), len(ls)+1):
#   comb(ls, it)

In [ ]:
get_evaluation(bm25,df_essay)
get_evaluation(bm25,df_essay_bard)

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7828308335199505
map = 0.4266597603598687
ndcg@10 = 0.25839946564971383


pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.8011600944064944
map = 0.4555596503068554
ndcg@10 = 0.26905775068663207


In [ ]:
topics = dataset.get_topics()
get_evaluation(bm25 >> pt.rewrite.RM3(index) >>bm25,topics)

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

pt.Experiment:   0%|          | 0/1 [00:00<?, ?system/s]

recall@1000 = 0.7735730861969958
map = 0.4380073771472925
ndcg@10 = 0.23487627395424318


In [ ]:
# def get_evaluation(br, topics):
#   res = pt.GridSearch(
#     br,
#     {br: {"bm25.b"  : np.arange(0.1,1.1,0.1),
#             "bm25.k_1": np.arange(0.1,5.2,0.2),
#     }},
#     topics,
#     qrels,
#     "map",
#     #jobs = 10,
#     verbose = True)
#   res_10 = pt.GridSearch(
#     br % 10,
#     {br: {"bm25.b"  : np.arange(0.1,1.1,0.1),
#             "bm25.k_1": np.arange(0.1,5.2,0.2),
#     }},
#     topics,
#     qrels,
#     nDCG,
#     #jobs = 10,
#     verbose = True,)

#   # res_100 = pt.Experiment(
#   #     [br % 100],
#   #     topics,
#   #     qrels,
#   #     baseline=0,
#   #     verbose=True,
#   #     eval_metrics=[RR, "map", nDCG])
#   res_1000 = pt.GridSearch(
#     br % 1000,
#     {br: {"bm25.b"  : np.arange(0.1,1.1,0.1),
#             "bm25.k_1": np.arange(0.1,5.2,0.2),
#     }},
#     topics,
#     qrels,
#     "recall",
#     #jobs = 10,
#     verbose = True,)

#   # print(res)
#   # print(f'The value of the evaluation parameters are as follows:\n 1) MRR@10 = {res_10.loc[0, "RR"]}\n 2) MRR@100 = {res_100.loc[0, "RR"]}\n 3) MRR@1000 = {res_1000.loc[0, "RR"]}\n 4) MAP@10 = {res_10.loc[0, "map"]}\n 5) MAP@100 = {res_100.loc[0, "map"]}\n 6) MAP@1000 = {res_1000.loc[0, "map"]}\n 7) nDCG@10 = {res_10.loc[0, "nDCG"]}\n 8) nDCG@100 = {res_100.loc[0, "nDCG"]} \n 9) nDCG@1000 = {res_1000.loc[0, "nDCG"]}')
#   print(res)
#   print(res_10)
#   print(res_1000)